In [ ]:
import torch
import e3cnn.nn as enn
import e3cnn.gspaces as gspaces

gspace = gspaces.rot3dOnR3()

in_type  = enn.FieldType(gspace, [gspace.irrep(0)])
out_type = enn.FieldType(gspace, [gspace.irrep(0), gspace.irrep(1), gspace.irrep(2)])

model = enn.R3Conv(in_type, out_type, kernel_size=5)

In [ ]:
cube = gspaces.octaOnR3()

In [ ]:
import numpy as np
def kernel_so3(L: int) -> np.ndarray:
    dims = [ 2 * l + 1 for l in range(L+1)]
    V = np.concatenate([np.eye(d).flatten() * np.sqrt(d) for d in dims])
    return V

def kernel_sphere(gspace: gspaces.GSpace3D, L: int) -> np.ndarray:
    sphere = gspace.fibergroup.homspace((False, -1))
    identity = gspace.fibergroup.identity
    return np.concatenate([
        sphere.basis(identity, (l,), (0,)).flatten() * np.sqrt(2*l+1)
        for l in range(L+1)
    ])

In [ ]:
dims = [ 2 * l + 1 for l in range(1+1)]
V = np.concatenate([np.eye(d).flatten() for d in dims])

In [ ]:
V

In [ ]:
kernel = kernel_sphere(gspace, 1)

grid = gspace.fibergroup.sphere_grid('tetra')

rho = gspace.fibergroup.bl_quotient_representation(1, (False,-1))

In [ ]:
kernel = kernel_so3(1)

grid = gspace.fibergroup.grid('tetra')

rho = gspace.fibergroup.bl_regular_representation(1)

In [ ]:
A = np.stack([ kernel @ rho(g).T for g in grid ])
# A /= np.sqrt(len(A))

In [ ]:
A

In [ ]:
A /= np.sqrt(len(A))

In [ ]:
A

In [ ]:
np.set_printoptions(precision=2)
np.set_printoptions(suppress=True)

In [ ]:
(A @ A.T)

In [ ]:
k @ gspace.fibergroup.bl_quotient_representation(2, (False,-1))(grid[-5]).T.round()

In [ ]:
cube.fibergroup.cube_vertices_representation

In [ ]:
[x.shape for name, x in list(model.basisexpansion.named_buffers())]

In [ ]:
model.basisexpansion.dimension()

In [ ]:
for info in model.basisexpansion.get_basis_info():
    print(f"{info['in_irrep'][0]} -> {info['out_irrep'][0]} at radius {info['radius']} shape: {info['shape']}")

In [ ]:
[(name, list(x.shape)) for name, x in list(model.named_parameters())]

In [ ]:
import sys
sys.path.append('..')
from data_utils.dataset import AsocaClassificationDataset, AsocaSegmentationDataset

In [ ]:
ds_seg = AsocaSegmentationDataset(ds_path='../dataset/processed/')

In [ ]:
ds = AsocaClassificationDataset(ds_path='../dataset/classification//')

In [ ]:
len(ds)

In [ ]:
ds[-1][0].shape

In [ ]:
from data_utils.datamodule import AsocaClassificationDataModule

In [ ]:
adm = AsocaClassificationDataModule(data_dir='../dataset/classification/')

In [ ]:
dl = adm.train_dataloader()

In [ ]:
next(iter(dl))

In [ ]:
adm.val_dataloader().dataset.file_ids

In [ ]:
def get_req(batch_size, channels):
    n_params = sum([
        batch_size * channels * 32**3,
        batch_size * channels * 32**3,
        batch_size * channels * 14**3,
        batch_size * channels * 14**3,
        batch_size * channels * 5**3,
        batch_size * channels * 5**3,
        512*2
    ])
    x = n_params * 2 # forward + backward
    x = x * 32 # 32 bit floats
    x = x / 8  # in bytes
    x = x / 1024**3 # in GB
    return x, n_params

In [ ]:
get_req(32, 480)

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
from train import AsocaClassificationConfig, AsocaSegmentationConfig

In [ ]:
config = AsocaClassificationConfig((68,68,68))

In [ ]:
config_seg = AsocaSegmentationConfig((68,68,68), patch_stride=(60,60,60))

In [ ]:
config_seg

In [ ]:
config

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def kernel_so3(L: int) -> np.ndarray:
    dims = [ 2 * l + 1 for l in range(L+1)]
    V = np.concatenate([np.eye(d).flatten() * np.sqrt(d) for d in dims])
    return V

def kernel_sphere(gspace: gspaces.GSpace3D, L: int) -> np.ndarray:
    sphere = gspace.fibergroup.homspace((False, -1))
    identity = gspace.fibergroup.identity

    return np.concatenate([
        sphere.basis(identity, (l,), (0,)).flatten() * np.sqrt(2*l+1)
        for l in range(L+1)
    ])

In [ ]:
gspace

In [ ]:
max_freq = 1

In [ ]:
sph_kernel = kernel_sphere(gspace, max_freq)
reg_kernel = kernel_so3(max_freq)

In [ ]:
sph_grid = gspace.fibergroup.sphere_grid(type='thomson_cube', N=2)
reg_grid = gspace.fibergroup.grid(type='thomson_cube', N=2)

In [ ]:
sph_rho = gspace.fibergroup.bl_quotient_representation(max_freq, (False,-1))
reg_rho = gspace.fibergroup.bl_regular_representation(max_freq)

In [ ]:
reg_grid.__len__()

In [ ]:
reg_grid

In [ ]:
plt.imshow(reg_rho(reg_grid[1]))

In [ ]:
sph_rho(sph_grid[0])